In [ ]:
from torch import nn, optim
from torch.nn import functional as F
import torch
from torch import nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import numpy as np
import  pandas as pd
import os
from scipy import sparse
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook as tqdm

import seaborn as sn
sn.set()

import sys
import warnings; 
warnings.simplefilter('ignore')

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

In [ ]:
DATA_DIR = '../input/netflixprizedataset'
VAE_path = 'np_VAE02.pt'
DAE_path = 'np_DAE.pt'
VAE_metrics_path = 'history_NP_beta02.csv'
DAE_metrics_path = 'history_NP_DAE.csv'

In [ ]:
train_data = pd.read_csv(DATA_DIR + '/train.csv')
val_data = pd.read_csv(DATA_DIR + '/val.csv')
test_data = pd.read_csv(DATA_DIR + '/test.csv')

In [ ]:
nItems = train_data.sid.nunique()
nItems

In [ ]:
train_data = sparse.csr_matrix((np.ones_like(train_data.uid), (train_data.uid.values, train_data.sid.values)), 
                             dtype='float64',
                             shape=(train_data.uid.nunique(),nItems))



val_data = sparse.csr_matrix((np.ones_like(val_data.uid), (val_data.uid.values, val_data.sid.values)), 
                             dtype='float64',
                             shape=(val_data.uid.nunique(), nItems))

test_data = sparse.csr_matrix((np.ones_like(test_data.uid), (test_data.uid.values, test_data.sid.values)), 
                             dtype='float64',
                             shape=(test_data.uid.nunique(), nItems))

In [ ]:
class netflixDataset(torch.utils.data.Dataset):
    def __init__(self, scr_matrix, eval = False,prop=0.2):
        self.scr_matrix = scr_matrix
        self.eval = eval
    
        self.prop = prop
      
          
    def __getitem__(self, idx):
      
      item = {}
        
      

      if self.eval:
        u_items = self.scr_matrix[idx,:].toarray()[0]
        
        nu_items = u_items.sum()       
        val_size = int(nu_items*self.prop)
        idx_labels = np.where(u_items == 1)[0]
        data = np.ones_like(u_items)
        
        
                
        val_idx = np.random.choice(idx_labels, size=val_size, replace=False)                   
        data[val_idx] = 0
         
        
        
        
        item['data'] = torch.tensor(u_items*data,dtype=torch.float64)     
        
        item['ground_truth'] = torch.tensor(np.logical_not(data),dtype=torch.float64)             
        
        
       
      else:
        item['data'] = torch.tensor(self.scr_matrix[idx,:].toarray(),dtype=torch.float64)
      return item
        

    def __len__(self):
        return self.scr_matrix.shape[0]

In [ ]:
class VAE(nn.Module):
    def __init__(self,n_Items, hidden=600, dimz= 200, p=0.5):
        super(VAE, self).__init__()

        self.n_Items = n_Items
        self.dimz = dimz
        self.hidden = hidden
        self.p = p

        self.inference = nn.Sequential(
           
            nn.Dropout(self.p),
            nn.Linear(self.n_Items,self.hidden),
            nn.Tanh(),
            nn.Linear(self.hidden,2*self.dimz)          
        )
        self.generative = nn.Sequential(
            nn.Linear(self.dimz,self.hidden),
            nn.Tanh(),
            nn.Linear(self.hidden,self.n_Items),
            
        )
  
        

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        
        return mu + std*eps* ( 1 if self.Mode =='train' else 0)


    def forward(self, x,Mode='train'):       
        self.Mode = Mode
        x = F.normalize(x, p=2, dim=1)  
        distribution = self.inference(x)



        mu, logvar = distribution[:, :self.dimz], distribution[:, self.dimz:]
        z = self.reparameterize(mu, logvar)
        logit = self.generative(z)

        
        return logit, mu, logvar

class DAE(nn.Module):
    def __init__(self,n_Items, dimz= 200, p=0.5):
        super(DAE, self).__init__()

        self.n_Items = n_Items
        self.dimz = dimz        
        self.p = p

        self.net = nn.Sequential(
           
            nn.Dropout(self.p),
            nn.Linear(self.n_Items,self.dimz),
            nn.Tanh(),          
            nn.Linear(self.dimz,self.n_Items),
        )
       
  
        

 
    def forward(self, x,Mode='train'):       
        self.Mode = Mode
        x = F.normalize(x, p=2, dim=1)  
        
        logit = self.net(x)        
        return logit

In [ ]:
def loss_function(recon_x, x, mu, logvar,N_i,beta):

   # BCE = F.binary_cross_entropy(recon_x.view(-1,n_items), x.view(-1,n_items), reduction='sum')
    
    LL = -torch.mean(torch.sum(F.log_softmax(recon_x, -1) * x, -1))
    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
   
    return LL + beta*KLD 

In [ ]:
def NDCG_at_k(labels, scores, k = 100):
    device = scores.device
    arg_sort_scores = torch.argsort(scores,1,descending=True)
    arg_sort_labels = torch.argsort(labels,1,descending=True)


    pred_labels = torch.gather(labels,1,arg_sort_scores[:,:k]).to(device)


    tp = (1. /torch.log(torch.arange(2,2+k).float())).to(device)


    dcg = (tp * pred_labels).sum(axis = 1)

    idcg = torch.Tensor([tp[:min(int(n),k)].sum() for n in labels.sum(1)]).to(device)

    ndcg = (dcg/idcg).mean()

    return ndcg

In [ ]:
def Recall_at_k(labels, scores, k = 20):
    device = scores.device
    arg_sort_scores = torch.argsort(scores,1,descending=True)
    arg_sort_labels = torch.argsort(labels,1,descending=True)

    pred_labels = torch.gather(labels,1,arg_sort_scores[:,:k]).to(device)

#     denominator = torch.Tensor([min(M, k) for M in labels.sum(1)]).to(device)
    denominator = labels.sum(1)
    denominator[denominator > k] = k

    return (pred_labels.sum(1) / denominator).mean()

In [ ]:
# Declare Model
model = VAE(nItems)
optimizer = optim.AdamW(model.parameters(), lr=1e-3,weight_decay=0.01)
model.to(device)

# load check point 
checkpoint = torch.load('../input/train-netflixprize/msd-beta1.pt')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']

# KL-Annealing for new training
anneal = 0
anneal_cap = checkpoint['beta']
anneal_steps = 1/200_000


cur_metric = -np.inf


# prepare Data
train_ds = netflixDataset(train_data)
train_dl = DataLoader(train_ds, batch_size=512)

val_ds = netflixDataset(val_data,eval=True)
val_dl = DataLoader(val_ds, batch_size=1024)



In [ ]:
epoch,anneal_cap,checkpoint['evaluate']['ndcg@100']

In [ ]:
def evaluate(model, val_loader, ndcg_k = [100], recall_k = [20,50],DAE = False):
    '''Evaluate model at Recall and NDCG metrics'''
    '''Return a dict with metrics'''
    metrics = {}
    for k in ndcg_k:
        metrics[f'ndcg@{k}'] = []
    for k in recall_k:
        metrics[f'recall@{k}'] = []
    # eval phase
    
    for data in val_loader:
        X = data['data'].float().to(device)  
        X = X.squeeze(1)
        
        ground_truth = torch.stack([data['ground_truth'][i,:] for i in range(X.shape[0])])\
                      .squeeze(1).to(device)
        if DAE:
            
            pred = model(X,Mode ='eval')
        else:
            pred = model(X,Mode ='eval')[0]

        pred = pred.detach()


        pred[X!=0] = -np.inf
        for k in ndcg_k:
            ndcg = NDCG_at_k(ground_truth,pred, k)
            metrics[f'ndcg@{k}'] .append(ndcg.item())
        for k in recall_k:
            recall = Recall_at_k(ground_truth,pred, k)
            metrics[f'recall@{k}'] .append(recall.item())
    # cal metrics
    for k in ndcg_k:
        metrics[f'ndcg@{k}'] = np.mean(metrics[f'ndcg@{k}'])
    for k in recall_k:
        metrics[f'recall@{k}'] = np.mean(metrics[f'recall@{k}'])
    return metrics


In [ ]:
# n_Epochs = 100

# train_dl = DataLoader(train_ds, batch_size=512,shuffle = True)
# pbar = tqdm(range(n_Epochs),total = n_Epochs)
# for epoch in pbar:
#     train_loss =  []
#     # train phase
#     model.train()
#     train_bar = tqdm(train_dl,total = len(train_dl))
#     for data in train_bar:
#         x = data['data'].float().to(device)
#         x = x.squeeze(1)
#         optimizer.zero_grad()  


#         recon_x, mu, logvar = model(x)   

#         CE = -torch.mean(torch.sum(F.log_softmax(recon_x, 1) * x, -1))
#         KLD = -0.5 * torch.mean(torch.sum(1 + logvar - mu.pow(2) - logvar.exp(), dim=1))

#         loss =  CE + anneal * KLD

#         # loss = loss_function(recon_batch, x, mu, logvar,n_items,anneal)   
#         #training on best beta
#         anneal = min(anneal+anneal_steps,anneal_cap)   
#         loss.backward()      

#         optimizer.step()
#         train_loss.append(loss.item())
#         metrics = {}
#         metrics['train_loss'] = loss.item()
#         train_bar.set_postfix(metrics)
#     # Eval phases
#     model.eval()
#     metrics = evaluate(model, val_dl)
#     metrics['train_loss'] = np.mean(train_loss)

#     ndcg = list(metrics.values())[0]
#     if ndcg > cur_metric:
#         cur_metric = ndcg
#         torch.save({
#                 'epoch': epoch,
#                 'model_state_dict': model.state_dict(),
#                 'optimizer_state_dict': optimizer.state_dict(),
# #                 'loss': metrics['loss'],
#                 'evaluate': metrics,
#                 'beta': anneal
#                 }, VAE_path)
#     pbar.set_postfix(metrics)
#     # write metrics to file
#     s = ['{:.3f}'.format(v) for v in metrics.values()]
#     with open(VAE_metrics_path, 'a') as f:
#         f.write(','.join(s) + '\n')
   

In [ ]:
# torch.save({
#                 'epoch': epoch,
#                 'model_state_dict': model.state_dict(),
#                 'optimizer_state_dict': optimizer.state_dict(),
# #                 'loss': metrics['loss'],
#                 'evaluate': metrics,
#                 'beta': anneal
#                 }, 'VAE_last_epoch.pt')

In [ ]:
# Declare Model
model = DAE(nItems)
optimizer = optim.AdamW(model.parameters(), lr=1e-3,weight_decay=0.01)
model.to(device)



cur_metric = -np.inf


# prepare Data
train_ds = netflixDataset(train_data)
train_dl = DataLoader(train_ds, batch_size=512)

val_ds = netflixDataset(val_data,eval=True)
val_dl = DataLoader(val_ds, batch_size=1024)

In [ ]:
n_Epochs = 100
cur_metric = -np.inf
train_dl = DataLoader(train_ds, batch_size=512,shuffle = True)
pbar = tqdm(range(n_Epochs),total = n_Epochs)
for epoch in pbar:
    train_loss =  []
    # train phase
    model.train()
    train_bar = tqdm(train_dl,total = len(train_dl))
    for data in train_bar:
        x = data['data'].float().to(device)
        x = x.squeeze(1)
        optimizer.zero_grad()  


        recon_x = model(x)   

        loss = -torch.mean(torch.sum(F.log_softmax(recon_x, 1) * x, -1))
           
        loss.backward()      

        optimizer.step()
        train_loss.append(loss.item())
        metrics = {}
        metrics['train_loss'] = loss.item()
        train_bar.set_postfix(metrics)
    # Eval phases
    model.eval()
    metrics = evaluate(model, val_dl,DAE=True)
    metrics['train_loss'] = np.mean(train_loss)

    ndcg = list(metrics.values())[0]
    if ndcg > cur_metric:
        cur_metric = ndcg
        torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
#                 'loss': metrics['loss'],
                'evaluate': metrics,
              
                }, DAE_path)
    pbar.set_postfix(metrics)
    # write metrics to file
    s = ['{:.3f}'.format(v) for v in metrics.values()]
    with open(DAE_metrics_path, 'a') as f:
        f.write(','.join(s) + '\n')

In [ ]:
torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
#                 'loss': metrics['loss'],
                'evaluate': metrics,
               
                }, 'DAE_last_epoch.pt')